In [1]:
import warnings
warnings.filterwarnings("ignore")
import nltk, re, pickle
from sklearn.externals import joblib
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
import pickle as pkl
import os, string
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.metrics import f1_score, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

In [2]:
def readData(dataFilePath):
    textData = open(dataFilePath, "r")
    textDataset = []    
    for line in textData.readlines():
        line = line.strip().split("\t")
        textDataset.append(line)
    return textDataset
        
dataFilePath = "Data/newData.txt"
textDataSet = readData(dataFilePath)
textDataFrame = pd.DataFrame(textDataSet, columns=['text', 'label'])
textDataFrame.tail()

,text,label
11536,@someUSER im going to bali tomorrow for the we...,Neutral
11537,@someUSER i guess we'll see how far we've come...,Positive
11538,"bola hai admin , aankhe kholo duites , kno...",Neutral
11539,get your sweat suits ! it's november and its ...,Positive
11540,maa sab samajti hein c> mom always know what y...,Neutral


In [3]:
xData = list(textDataFrame.text)
xLabel = list(textDataFrame.label)
print(len(xData), len(xLabel))

11541 11541


In [4]:
contractionMapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                       "here's": "here is",
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have" }

stopWords = set(stopwords.words('english'))
acronyms = {}
extraAcronyms = ["'t", 'not', 'nt', 'not', 'sec', 'seconds', "'d", 'had', "'ve", 'have', "'ll", "will", "'s", 'is', 'af', 'as fuck', 'fb', 'facebook', 'C', 'see','u', 'you', 'gd', 'good', 'n8', 'night', 'LOL','Laughing Out Loud','BRB','Be Right Back','BBL','Be Back Later','ASAP','As Soon As Possible','B4','Before','B4N','Bye For Now','BTW','By The Way','B/C', 'Because','C U L8R','See You Later','Def','Definitely','tmrw','Tomorrow','ETA','Estimated Time of Arrival','F2F','Face To Face', 'GF', 'Girlfriend', 'BF', 'Boyfriend','BFF','Best Friend Forever','AML','All My Love','GTG', 'Got To Go', 'G2G','Got To Go', 'GR8', 'Great', 'IDK', 'I Don’t Know', 'IDC', 'I Don’t Care', 'ILY', 'I Love You', 'ILU','I Love You','JK','Just Kidding','K','OK','LMK','Let Me Know','THX','Thanks','PLS','Please','TTYL','Talk To You Later','TMI','Too Much Information','CYE','Check Your Email','FYI','For Your Information','IMHO','In My Honest Opinion','ROFL','Rolling On the Floor Laughing','BION','Believe It Or Not','BRT','Be Right There','Y','Why?','PCM','Please Call Me','IYKWIM','If You Know What I Mean','WYSIWYG','What You See Is What You Get','XOXO','Hugs and Kisses', 'ur', 'your', 'tmrw', 'tomorrow', 'omg', 'oh my god', 'ab', 'about', 'abt', 'about', 'adventuritter', 'adventurous', 'b', 'be', 'beetweet', 'hot tweet', 'BFN', 'bye for now', 'bgd', 'background', 'BR', 'best regards', 'BTW', 'by the way', 'chk', 'check', 'cld', 'could', 'clk', 'click', 'cre8', 'create', 'da', 'the', 'deets', 'details',  'dm', 'direct message', 'egotwistical', 'egotistical and Twitter', 'EM', 'email', 'eml', 'e-mail', 'EMA', 'e-mail address', 'F2F', 'face to face', 'fab', 'fabulous', 'FAV', 'favorite', 'FOMO', 'fear of missing out', 'FTL', 'for the loss', 'FTW', 'for the win', 'IC', 'I see', 'ICYMI', 'in case you missed it', 'MRT', 'modified retweet', 'MTF', 'more to follow', 'NTS', 'note to self', 'OH', 'overheard', 'SMH', 'shaking my head', 'SP', 'Sponsored', 'TBH', 'to be honest', 'TBT', 'Throw Back Thursday', 'TFTF', 'thanks for the follow', 'TMB', 'Tweet me back', 'woz',  'was', 'wtv',  'whatever', 'ykyat', 'you know you’re addicted to', 'YOLO', 'you only live once', 'YOYO', 'you’re on your own']
c=0
for i in range(0, len(extraAcronyms)-1, 2):
    key = extraAcronyms[c].lower()
    value = extraAcronyms[c+1].lower()
    acronyms[key] = value
    c+=2
acronyms.update(contractionMapping)
listAcronyms = list(acronyms)
joblib.dump(acronyms, 'Pickles/contractionMapping.pkl')

['contractionMapping.pkl']

In [5]:
def findAcronyms(acronym):
    if(acronym in listAcronyms):
        return acronyms[acronym]
    else:
        return acronym

def dataPreprocess(data):
    preprocessedSentence = []
    for sentence in data:
        newTokens = []
        removetable = str.maketrans('', '', string.punctuation)
        removetable1 = str.maketrans('', '', string.digits)
        wordsOfSentence = sentence.split()
        for word in wordsOfSentence:
            word = word.lower()
            word = findAcronyms(word)
            word = word.translate(removetable)
            word = word.translate(removetable1)
            if(word.startswith('@')!=True and word.startswith('<')!=True and word.startswith('www')!=True and word.startswith('http')!=True and word.endswith('com')!=True):
                if(len(word)>1):
                    word = re.sub(r'(.)\1+', r'\1\1', word)
                    newTokens.append(word)
        if(len(newTokens)==0):
            wordsOfSentence = [x.lower() for x in wordsOfSentence]
            preprocessedSentence.append(wordsOfSentence)
            #print(wordsOfSentence)
        else:
            preprocessedSentence.append(newTokens)
    return preprocessedSentence

textFeaturesOld = dataPreprocess(xData)
textFeatures_ = [' '.join(i) for i in textFeaturesOld]
textFeatures = np.asarray(textFeatures_)
labels = np.asarray(xLabel)
print(textFeatures.shape, labels.shape)

(11541,) (11541,)


In [6]:
#No of samples in each class

counter = Counter(labels)
print(counter)

Counter({'Neutral': 4529, 'Positive': 4239, 'Negative': 2773})


In [7]:
#Data split into train and test sets

xTrain, xValid, xTrainLabels, xValidLabels = model_selection.train_test_split(textFeatures, labels, test_size=0.05, random_state=11)
tfidfVectorObject = TfidfVectorizer(analyzer='word', max_features=5000)
tfidfVectorObject.fit(textFeatures)
joblib.dump(tfidfVectorObject, 'Pickles/tfidfFeatureVector.pkl')
xTrainTfidfVectors =  tfidfVectorObject.transform(xTrain).toarray()
xValidTfidfVectors =  tfidfVectorObject.transform(xValid).toarray()
print(xTrainTfidfVectors.shape, xValidTfidfVectors.shape)

(10963, 5000) (578, 5000)


In [8]:
#Results

def trainModel(classifier, trainFetureVector, trainLabel, validFeatureVector):
    classifier.fit(trainFetureVector, trainLabel)
    joblib.dump(classifier, 'Pickles/mlpCodeMixedModel.pkl')
    predictedLabels = classifier.predict(validFeatureVector)
    f1Score = f1_score(xValidLabels, predictedLabels, average='macro')
    print("f1-score macro: ",f1Score)
    print(classification_report(xValidLabels, predictedLabels))
    return classifier

In [9]:
#Multi Layer Perceptron Before Data Balancing

mlp = MLPClassifier(hidden_layer_sizes=(300,), max_iter=3, solver='adam', learning_rate_init=0.001, verbose=10, random_state=1)
classifier = trainModel(mlp, xTrainTfidfVectors, xTrainLabels, xValidTfidfVectors)

Iteration 1, loss = 1.03836361
Iteration 2, loss = 0.81351422
Iteration 3, loss = 0.62702347
f1-score macro:  0.6206214233358214
              precision    recall  f1-score   support

    Negative       0.60      0.51      0.55       134
     Neutral       0.67      0.69      0.68       241
    Positive       0.62      0.65      0.63       203

    accuracy                           0.63       578
   macro avg       0.63      0.62      0.62       578
weighted avg       0.63      0.63      0.63       578



In [10]:
#Data Classes Balancing

from imblearn.over_sampling import RandomOverSampler
ROS = RandomOverSampler(random_state=11)
overSampledTfidfText, overSampledTfidfLabels = ROS.fit_sample(xTrainTfidfVectors, xTrainLabels)
print(overSampledTfidfText.shape, overSampledTfidfLabels.shape)
print(Counter(overSampledTfidfLabels))

Using TensorFlow backend.


(12864, 5000) (12864,)
Counter({'Positive': 4288, 'Neutral': 4288, 'Negative': 4288})


In [15]:
#Multi Layer Perceptron After Data Balancing

mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=3, solver='adam', learning_rate_init=0.001, verbose=10, random_state=1)
classifier = trainModel(mlp, overSampledTfidfText, overSampledTfidfLabels, xValidTfidfVectors)

Iteration 1, loss = 1.04788366
Iteration 2, loss = 0.84274815
Iteration 3, loss = 0.66609193
f1-score macro:  0.6471867119832249
              precision    recall  f1-score   support

    Negative       0.59      0.63      0.61       134
     Neutral       0.68      0.69      0.68       241
    Positive       0.68      0.62      0.65       203

    accuracy                           0.65       578
   macro avg       0.65      0.65      0.65       578
weighted avg       0.66      0.65      0.65       578

